In [102]:
from timeit import default_timer
start = default_timer()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [103]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
!mkdir -p ~/.kaggle

In [105]:
!cp kaggle.json ~/.kaggle/

In [106]:
!chmod 700 ~/.kaggle/kaggle.json

In [107]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [108]:
!unzip movielens-20m-dataset.zip

Archive:  movielens-20m-dataset.zip
replace genome_scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: genome_scores.csv       
  inflating: genome_tags.csv         
  inflating: link.csv                
  inflating: movie.csv               
  inflating: rating.csv              A

  inflating: tag.csv                 


In [109]:
import pandas as pd
tag = pd.read_csv("/content/tag.csv")
movies = pd.read_csv("/content/movie.csv")
rating = pd.read_csv("/content/rating.csv")
link = pd.read_csv("/content/link.csv")
genome_tags = pd.read_csv("/content/genome_tags.csv")
genome_scores = pd.read_csv("/content/genome_scores.csv")

In [1]:
!apt-get update
# Download Java Virtual Machine (JVM)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [972 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages

In [2]:
# Download Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# Unzip the file
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [3]:
!pip install -q findspark

In [4]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.2.1-bin-hadoop3.2'

In [5]:
!ls

kaggle.json  spark-3.2.1-bin-hadoop3.2
sample_data  spark-3.2.1-bin-hadoop3.2.tgz


In [6]:
import findspark
findspark.init()

In [7]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=83414b08a97c1e4be90071a6c3806d935b85a7ec3a42614652fa4b8151f9e713
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [126]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, collect_list, split, explode, count, row_number
from pyspark.sql.window import Window
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sc

<SparkContext master=local[*] appName=pyspark-shell>

### To Check if Spark is active

In [10]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Importing the required libraries

In [11]:
import os
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import pandas as pd
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()

### Converting the data into Spark DataFrames

In [18]:
movie_df=spark.read.csv('/content/movie.csv',inferSchema=True,header=True)
link_df=spark.read.csv('/content/link.csv',inferSchema=True,header=True)
rating_df=spark.read.csv('/content/rating.csv',inferSchema=True,header=True)
tag_df=spark.read.csv('/content/tag.csv',inferSchema=True,header=True)

In [19]:
# Movies dataframe
movie_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [20]:
# Link dataframe
link_df.show()

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [21]:
#Tags Dataframe
tag_df.show()

+------+-------+-----------------+-------------------+
|userId|movieId|              tag|          timestamp|
+------+-------+-----------------+-------------------+
|    18|   4141|      Mark Waters|2009-04-24 18:19:40|
|    65|    208|        dark hero|2013-05-10 01:41:18|
|    65|    353|        dark hero|2013-05-10 01:41:19|
|    65|    521|    noir thriller|2013-05-10 01:39:43|
|    65|    592|        dark hero|2013-05-10 01:41:18|
|    65|    668|        bollywood|2013-05-10 01:37:56|
|    65|    898| screwball comedy|2013-05-10 01:42:40|
|    65|   1248|    noir thriller|2013-05-10 01:39:43|
|    65|   1391|             mars|2013-05-10 01:40:55|
|    65|   1617|         neo-noir|2013-05-10 01:43:37|
|    65|   1694|            jesus|2013-05-10 01:38:45|
|    65|   1783|    noir thriller|2013-05-10 01:39:43|
|    65|   2022|            jesus|2013-05-10 01:38:45|
|    65|   2193|           dragon|2013-05-10 02:01:54|
|    65|   2353|conspiracy theory|2013-05-10 02:01:06|
|    65|  

In [22]:
#Ratings Dataframe
rating_df.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|   4.0|2004-09-10 03:08:54|
|     1|    223|   4.0|2005-04-02 23:46:13|
|     1|    253|   4.0|2005-04-02 23:35:40|
|     1|    260|   4.0|2005-04-02 23:33:46|
|     1|    293|   4.0|2005-04-02 23:31:43|
|     1|    296|   4.0|2005-04-02 23:32:47|
|     1|    318|   4.0|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|   4.0|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|   3.0|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

### Correlation of the data

In [23]:
def correlation_matrix(df, corr_columns, method='pearson'):

    vector_col = "corr_features"
    assembler = VectorAssembler(inputCols=corr_columns, outputCol=vector_col)
    df_vector = assembler.transform(df).select(vector_col)
    matrix = Correlation.corr(df_vector, vector_col, method)

    result = matrix.collect()[0]["pearson({})".format(vector_col)].values
    return pd.DataFrame(result.reshape(-1, len(corr_columns)), columns=corr_columns, index=corr_columns)

In [24]:
# Correlation of ratings dataset
correlation_matrix(rating_df,['userId','movieId','rating'])

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


,userId,movieId,rating
userId,1.000000,-0.000850,0.001175
movieId,-0.000850,1.000000,0.002606
rating,0.001175,0.002606,1.000000


In [25]:
# Correlation of Tags dataset
correlation_matrix(tag_df,['userId','movieId'])

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


,userId,movieId
userId,1.000000,-0.006086
movieId,-0.006086,1.000000


### Extracting the year and the genres for all movies

In [26]:
movie_df.select("title","genres").show()

+--------------------+--------------------+
|               title|              genres|
+--------------------+--------------------+
|    Toy Story (1995)|Adventure|Animati...|
|      Jumanji (1995)|Adventure|Childre...|
|Grumpier Old Men ...|      Comedy|Romance|
|Waiting to Exhale...|Comedy|Drama|Romance|
|Father of the Bri...|              Comedy|
|         Heat (1995)|Action|Crime|Thri...|
|      Sabrina (1995)|      Comedy|Romance|
| Tom and Huck (1995)|  Adventure|Children|
| Sudden Death (1995)|              Action|
|    GoldenEye (1995)|Action|Adventure|...|
|American Presiden...|Comedy|Drama|Romance|
|Dracula: Dead and...|       Comedy|Horror|
|        Balto (1995)|Adventure|Animati...|
|        Nixon (1995)|               Drama|
|Cutthroat Island ...|Action|Adventure|...|
|       Casino (1995)|         Crime|Drama|
|Sense and Sensibi...|       Drama|Romance|
|   Four Rooms (1995)|              Comedy|
|Ace Ventura: When...|              Comedy|
|  Money Train (1995)|Action|Com

In [27]:
# Testing and normally extracting the year
movie_df_title_yr = movie_df.select("title").collect()
final_yr = []
for i in movie_df_title_yr:
    final_yr.append(i[0][-5:-1])
final_yr[0:10]


['1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995',
 '1995']

In [28]:
#Extracting the year in the title column and showing it with genre

from pyspark.sql.functions import regexp_replace,col
from pyspark.sql.functions import udf
import re

def years(s):
    return s[-5:-1]

yr_udf = udf(years)

movie_df.withColumn("title", yr_udf("title")).show()

+-------+-----+--------------------+
|movieId|title|              genres|
+-------+-----+--------------------+
|      1| 1995|Adventure|Animati...|
|      2| 1995|Adventure|Childre...|
|      3| 1995|      Comedy|Romance|
|      4| 1995|Comedy|Drama|Romance|
|      5| 1995|              Comedy|
|      6| 1995|Action|Crime|Thri...|
|      7| 1995|      Comedy|Romance|
|      8| 1995|  Adventure|Children|
|      9| 1995|              Action|
|     10| 1995|Action|Adventure|...|
|     11| 1995|Comedy|Drama|Romance|
|     12| 1995|       Comedy|Horror|
|     13| 1995|Adventure|Animati...|
|     14| 1995|               Drama|
|     15| 1995|Action|Adventure|...|
|     16| 1995|         Crime|Drama|
|     17| 1995|       Drama|Romance|
|     18| 1995|              Comedy|
|     19| 1995|              Comedy|
|     20| 1995|Action|Comedy|Cri...|
+-------+-----+--------------------+
only showing top 20 rows



# Spark-SQL Analysis

## Creating Views for spark dataframes

In [29]:
rating_df.createOrReplaceTempView("ratings")
movie_df.createOrReplaceTempView("movies")
tag_df.createOrReplaceTempView("tags")
link_df.createOrReplaceTempView("links")
# Movies with year separated from the title
movie_df.withColumn("title", yr_udf("title")).createOrReplaceTempView("movies_year")

### Query 1: How many number of Movies are there for each rating? 

In [30]:
spark.sql("SELECT rating,COUNT(movieID) as No_Of_Movies FROM ratings GROUP BY rating ORDER BY rating DESC").show()

+------+------------+
|rating|No_Of_Movies|
+------+------------+
|   5.0|     2898660|
|   4.5|     1534824|
|   4.0|     5561926|
|   3.5|     2200156|
|   3.0|     4291193|
|   2.5|      883398|
|   2.0|     1430997|
|   1.5|      279252|
|   1.0|      680732|
|   0.5|      239125|
+------+------------+



### Query 2: What are top 10 most rated movies?

In [31]:
query = "Select m.title,r.movieID,r.rating from ratings r, movies m where m.movieID = r.movieID order by r.rating desc limit 10"
spark.sql(query).show()

+--------------------+-------+------+
|               title|movieID|rating|
+--------------------+-------+------+
| Blade Runner (1982)|    541|   5.0|
|Amityville Horror...|   1327|   5.0|
|Terminator 2: Jud...|    589|   5.0|
|Jurassic Park (1993)|    480|   5.0|
|2001: A Space Ody...|    924|   5.0|
|Lord of the Rings...|   7153|   5.0|
|Star Wars: Episod...|   1196|   5.0|
|Mr. Holland's Opu...|     62|   5.0|
|Star Wars: Episod...|   1210|   5.0|
|Star Wars: Episod...|    260|   5.0|
+--------------------+-------+------+



### Query 3: How many users have rated each movie?

In [32]:
spark.sql("SELECT movieID, COUNT(userID) as No_Of_Users FROM ratings GROUP BY movieID ORDER BY COUNT(userID) DESC").show()

+-------+-----------+
|movieID|No_Of_Users|
+-------+-----------+
|    296|      67310|
|    356|      66172|
|    318|      63366|
|    593|      63299|
|    480|      59715|
|    260|      54502|
|    110|      53769|
|    589|      52244|
|   2571|      51334|
|    527|      50054|
|      1|      49695|
|    457|      49581|
|    150|      47777|
|    780|      47048|
|     50|      47006|
|   1210|      46839|
|    592|      46054|
|   1196|      45313|
|   2858|      44987|
|     32|      44980|
+-------+-----------+
only showing top 20 rows



### Query 4: What is the Total Rating for each movie? 

In [33]:
query = 'SELECT movieID, SUM(rating) as Total_Rating FROM ratings GROUP BY movieID ORDER BY movieID'
spark.sql(query).show()

+-------+------------+
|movieID|Total_Rating|
+-------+------------+
|      1|    194866.0|
|      2|     71444.0|
|      3|     40128.5|
|      4|      7886.0|
|      5|     37268.5|
|      6|     91651.0|
|      7|     43633.0|
|      8|      4446.0|
|      9|     11899.5|
|     10|     99488.0|
|     11|     66613.0|
|     12|     10073.0|
|     13|      4781.0|
|     14|     20668.0|
|     15|      7921.0|
|     16|     65879.0|
|     17|     82018.5|
|     18|     17553.0|
|     19|     54594.0|
|     20|     11765.0|
+-------+------------+
only showing top 20 rows



### Query 5: What is the Average Rating for each movie?

In [34]:
query = 'SELECT movieID, ROUND(AVG(rating),2) as Average_Rating FROM ratings GROUP BY movieID ORDER BY movieID'
spark.sql(query).show()

+-------+--------------+
|movieID|Average_Rating|
+-------+--------------+
|      1|          3.92|
|      2|          3.21|
|      3|          3.15|
|      4|          2.86|
|      5|          3.06|
|      6|          3.83|
|      7|          3.37|
|      8|          3.14|
|      9|           3.0|
|     10|          3.43|
|     11|          3.67|
|     12|          2.62|
|     13|          3.27|
|     14|          3.43|
|     15|          2.72|
|     16|          3.79|
|     17|          3.97|
|     18|          3.37|
|     19|          2.61|
|     20|          2.88|
+-------+--------------+
only showing top 20 rows



### Query 6: How many movies are there for each genre?

In [35]:
spark.sql("Select genres, count(movieID) as Total_Movies from movies group by genres order by count(movieID) desc").show()

+--------------------+------------+
|              genres|Total_Movies|
+--------------------+------------+
|               Drama|        4520|
|              Comedy|        2294|
|         Documentary|        1942|
|        Comedy|Drama|        1264|
|       Drama|Romance|        1075|
|      Comedy|Romance|         757|
|Comedy|Drama|Romance|         605|
|              Horror|         565|
|         Crime|Drama|         448|
|      Drama|Thriller|         426|
|           Drama|War|         399|
|     Horror|Thriller|         374|
|Crime|Drama|Thriller|         304|
|            Thriller|         268|
|  (no genres listed)|         246|
|             Western|         215|
|        Action|Drama|         210|
|        Comedy|Crime|         187|
|              Action|         178|
|       Comedy|Horror|         170|
+--------------------+------------+
only showing top 20 rows



### Query 7: How many movies have been rated each year?

In [36]:
spark.sql("Select title as Year, count(movieID) as Total_Movies from movies_year group by title order by count(movieID) desc").show()

+----+------------+
|Year|Total_Movies|
+----+------------+
|2009|        1113|
|2012|        1021|
|2011|        1014|
|2013|        1010|
|2008|         978|
|2010|         958|
|2007|         900|
|2006|         854|
|2014|         740|
|2005|         739|
|2004|         706|
|2002|         678|
|2003|         655|
|2001|         632|
|2000|         613|
|1998|         554|
|1999|         542|
|1997|         528|
|1996|         509|
|1995|         474|
+----+------------+
only showing top 20 rows



### Query 8: Which were the least rated movies in the year of 2005?

In [37]:
query = "Select distinct(m.title),r.rating,m1.title as Year from movies m, ratings r, movies_year as m1 where m.movieID = r.movieID and m.movieID = m1.movieID and r.movieID = m1.movieId and m1.title = '2005' order by r.rating"
spark.sql(query).show()

+--------------------+------+----+
|               title|rating|Year|
+--------------------+------+----+
|  Jacket, The (2005)|   0.5|2005|
|   Dark Water (2005)|   0.5|2005|
| Transamerica (2005)|   0.5|2005|
|Trust the Man (2005)|   0.5|2005|
|Tristram Shandy: ...|   0.5|2005|
|Mr. & Mrs. Smith ...|   0.5|2005|
| Kid & I, The (2005)|   0.5|2005|
|Wayward Cloud, Th...|   0.5|2005|
|   C.R.A.Z.Y. (2005)|   0.5|2005|
|Bad News Bears (2...|   0.5|2005|
|Chronicles of Nar...|   0.5|2005|
|Proposition, The ...|   0.5|2005|
|       Sahara (2005)|   0.5|2005|
|Pacifier, The (2005)|   0.5|2005|
|Kingdom of Heaven...|   0.5|2005|
|       Cursed (2005)|   0.5|2005|
|  White Noise (2005)|   0.5|2005|
|Constant Gardener...|   0.5|2005|
|Legend of Zorro, ...|   0.5|2005|
|Dukes of Hazzard,...|   0.5|2005|
+--------------------+------+----+
only showing top 20 rows



### Query 9: What are the genres of the top 5 rated movies?

In [38]:
query = "Select m.title,m.genres,r.rating from ratings r, movies m where m.movieID = r.movieID order by r.rating desc limit 5"
spark.sql(query).show()

+--------------------+--------------------+------+
|               title|              genres|rating|
+--------------------+--------------------+------+
|       Freaks (1932)|  Crime|Drama|Horror|   5.0|
|Legends of the Fa...|Drama|Romance|War...|   5.0|
|Mr. Holland's Opu...|               Drama|   5.0|
|Lord of the Rings...|   Adventure|Fantasy|   5.0|
|From Dusk Till Da...|Action|Comedy|Hor...|   5.0|
+--------------------+--------------------+------+



### Query 10: Which are the top rated by the users Sci-fi movies?

In [39]:
query = "Select m.title,m.genres as Genre_SCIFI,r.rating from movies m,ratings r where genres Like '%Sci-Fi%' and m.movieID = r.movieID order by r.rating desc"
spark.sql(query).show()

+--------------------+--------------------+------+
|               title|         Genre_SCIFI|rating|
+--------------------+--------------------+------+
|Independence Day ...|Action|Adventure|...|   5.0|
|Star Trek: First ...|Action|Adventure|...|   5.0|
|        Alien (1979)|       Horror|Sci-Fi|   5.0|
|Star Trek: The Mo...|    Adventure|Sci-Fi|   5.0|
|           Pi (1998)|Drama|Sci-Fi|Thri...|   5.0|
|Star Trek VI: The...|Action|Mystery|Sc...|   5.0|
|2001: A Space Ody...|Adventure|Drama|S...|   5.0|
|Star Trek V: The ...|       Action|Sci-Fi|   5.0|
|Star Wars: Episod...|Action|Adventure|...|   5.0|
|Star Trek II: The...|Action|Adventure|...|   5.0|
| Donnie Darko (2001)|Drama|Mystery|Sci...|   5.0|
|Star Trek III: Th...|Action|Adventure|...|   5.0|
|Star Trek: Genera...|Adventure|Drama|S...|   5.0|
|Star Trek IV: The...|Adventure|Comedy|...|   5.0|
|E.T. the Extra-Te...|Children|Drama|Sc...|   5.0|
|  Judge Dredd (1995)| Action|Crime|Sci-Fi|   5.0|
|Forbidden Planet ...|        D

### Query 11: Which are the Worst Comedy movies rated by the users in the year 2012?

In [40]:
query = "Select distinct(m.title),m1.title as Year,m.genres as Genre_Comedy,r.rating from movies m, ratings r,movies_year as m1 where m.movieID = r.movieID and m.movieID = m1.movieID and r.movieID = m1.movieId and m.genres Like '%Comedy%' and m1.title = '2012' order by r.rating"
spark.sql(query).show()

+--------------------+----+--------------------+------+
|               title|Year|        Genre_Comedy|rating|
+--------------------+----+--------------------+------+
|What's in a Name ...|2012|              Comedy|   0.5|
|Thousand Words, A...|2012|        Comedy|Drama|   0.5|
|Dictator, The (2012)|2012|              Comedy|   0.5|
| Tai Chi Hero (2012)|2012|Action|Comedy|Dra...|   0.5|
|Cabin in the Wood...|2012|Comedy|Horror|Sci...|   0.5|
|Casa de mi Padre ...|2012|              Comedy|   0.5|
|Sleepwalk with Me...|2012|        Comedy|Drama|   0.5|
| Rock of Ages (2012)|2012|Comedy|Drama|Musi...|   0.5|
|That's My Boy (2012)|2012|              Comedy|   0.5|
|   Wanderlust (2012)|2012|              Comedy|   0.5|
|          LOL (2012)|2012|Comedy|Drama|Romance|   0.5|
|Wreck-It Ralph (2...|2012|    Animation|Comedy|   0.5|
|          Ted (2012)|2012|      Comedy|Fantasy|   0.5|
| Dark Shadows (2012)|2012|  Comedy|Horror|IMAX|   0.5|
|Journey 2: The My...|2012|Action|Adventure|...|

# Query 12: Find the number of users who watched the movie "Jumanji".

In [45]:
# register 'movie' and 'rating' dataframes as SQL temporary views
movie_df.createOrReplaceTempView("movie")
rating_df.createOrReplaceTempView("rating")

In [47]:
spark.sql("SELECT title, count(userId) AS total_viewers FROM movie INNER JOIN rating ON movie.movieId = rating.movieId WHERE title = 'Jumanji (1995)' GROUP BY title").show()


+--------------+-------------+
|         title|total_viewers|
+--------------+-------------+
|Jumanji (1995)|        22243|
+--------------+-------------+



# Query 13: Find the names of the movies that users described as "boring".

In [50]:
tag = tag_df.select((col('userId')),
                  col('movieId'),
                  lower(col('tag')).alias('lower_tag'),
                  col('timestamp'))

In [51]:
# execute inner join between 'movie' and 'tag' dataframes
joined = movie_df.join(tag, ["movieId"], 'inner')

In [101]:
# create new dataframe 'boring_movies' with the columns
# 'title' and 'lower_tag' from dataframe joined
# and only the rows, which contain the word 'boring' in the lower_tag column
boring_movies = joined.filter(joined['lower_tag'].contains('boring')).drop_duplicates(subset=['movieId']).select(joined['title'], joined['lower_tag'])


In [53]:
# register the 'boring_movies' dataframe as an SQL temporary view
boring_movies.createOrReplaceTempView("boring_movies")

In [55]:
query="SELECT * FROM boring_movies WHERE lower_tag NOT LIKE '% %' ORDER BY title ASC"
spark.sql(query).show(truncate=0)

+-------------------------------------+---------+
|title                                |lower_tag|
+-------------------------------------+---------+
|(500) Days of Summer (2009)          |boring   |
|101 Reykjavik (101 Reykjavík) (2000) |boring   |
|12 Years a Slave (2013)              |boring   |
|1408 (2007)                          |boring   |
|1492: Conquest of Paradise (1992)    |boring   |
|2001: A Space Odyssey (1968)         |boring   |
|2010: The Year We Make Contact (1984)|boring   |
|2046 (2004)                          |boring   |
|21 Grams (2003)                      |boring   |
|24 Hour Party People (2002)          |boring   |
|3-Iron (Bin-jip) (2004)              |boring   |
|6 Bullets (2012)                     |boring   |
|633 Squadron (1964)                  |boring   |
|7 Plus Seven (1970)                  |boring   |
|8 Women (2002)                       |boring   |
|A.I. Artificial Intelligence (2001)  |boring   |
|According to Greta (2009)            |boring   |


## Query 14:Find the number of users who have described a movie as "Bollywood" and they have rated it with a score > 3.


In [56]:
tag = tag_df.select((col('userId')),
                  col('movieId'),
                  lower(col('tag')).alias('lower_tag'),
                  col('timestamp'))

In [58]:
# execute inner join between 'rating' and 'tag' dataframes
joined = rating_df.join(tag, ['userId', 'movieId'], 'inner')


In [59]:
# create new dataframe 'bollywood_movies' with the columns
# 'userId', 'rating' and 'lower_tag' from dataframe joined
# and only the rows, which contain the word 'bollywood' in the lower_tag column
bollywood_movies = joined.filter(joined['lower_tag'].contains('bollywood')).drop_duplicates(subset=['userId']).select(joined['userId'], joined['rating'], joined['lower_tag'])


In [60]:
# register the 'bollywood_movies' dataframe as an SQL temporary view
bollywood_movies.createOrReplaceTempView("bollywood_movies")


In [61]:
query="SELECT * FROM bollywood_movies WHERE rating > 3.0 AND lower_tag NOT LIKE '%not%' ORDER BY userId ASC"
spark.sql(query).show()


+------+------+-------------------+
|userId|rating|          lower_tag|
+------+------+-------------------+
| 10573|   4.0|          bollywood|
| 19837|   5.0|          bollywood|
| 23333|   4.0|          bollywood|
| 25004|   5.0|          bollywood|
| 31338|   4.5|          bollywood|
| 33323|   3.5|          bollywood|
| 35170|   4.0|          bollywood|
| 40514|   5.0|          bollywood|
| 41165|   4.5|          bollywood|
| 48816|   4.5|bollywood influence|
| 51539|   4.0|          bollywood|
| 54900|   3.5|          bollywood|
| 63618|   3.5|          bollywood|
| 65908|   4.5|          bollywood|
| 70279|   4.5|          bollywood|
| 77137|   5.0|          bollywood|
| 86883|   4.0|          bollywood|
|106755|   4.0|bollywood influence|
|130827|   3.5|bollywood influence|
|131829|   4.0|          bollywood|
+------+------+-------------------+
only showing top 20 rows



## Query 15: Find the tags for each movie and the name of the movie before the year 2015.

In [62]:
# execute inner join between 'movie_df' and 'tag_df' dataframes
mxt = movie_df.join(tag_df, movie_df["movieId"]==tag_df["movieId"], "inner")\
        .select( tag_df["userId"],
                 tag_df["tag"],
                 movie_df["title"])

In [64]:
# keep only the rows which contain the year 2015
fift = mxt.filter(tag_df.timestamp.contains('2015-'))


In [68]:
query = fift.groupBy("title").agg(collect_list("tag").alias("tag")).orderBy("title").show(truncate=0)


+----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                   |tag                                                                                                                                                                                                                                                                                                                                                                                                     |
+----------------------------------------+--------------------------------------------------------------------------------------

## Query 16: Find the movies with the most ratings for each movie category.#

In [73]:
# because some movies have more than one genres listed,
# we split the column 'genres' with the '|' operator so we can process on each genre separately
# and then use explode so that every distinct genre will be in their own row
movie = movie_df.withColumn("genres", explode(split("genres", "[|]")))

In [75]:
# execute inner join between 'movie' and 'rating' dataframes
joined = movie.join(rating_df, ["movieId"], 'inner')

In [78]:
# create new dataframe 'top_movie_genre' with the columns of dataframe 'joined'
# grouped by the columns 'title' and 'genres'
# and count the number of ratings for each movie of each genre
top_movie_genre = joined.select('*').groupby(joined["title"], joined["genres"]).agg(count('rating').alias('total_ratings'))
# create a window partition over the 'genres' column
window = Window.partitionBy(top_movie_genre["genres"]).orderBy(top_movie_genre["total_ratings"].desc())

In [79]:
# give the sequential row number 1, to the result of each window partition.
# In that way, we find the rank 1 movie of each genre with the larger number of ratings
top = top_movie_genre.select('*', row_number().over(window).alias('rank')).filter(col('rank') <= 1)


In [80]:
query = top.select(top["genres"], top["title"], top["total_ratings"]).orderBy(top["genres"].asc()).show()

+------------------+--------------------+-------------+
|            genres|               title|total_ratings|
+------------------+--------------------+-------------+
|(no genres listed)|Doctor Who: The T...|           36|
|            Action|Jurassic Park (1993)|        59715|
|         Adventure|Jurassic Park (1993)|        59715|
|         Animation|    Toy Story (1995)|        49695|
|          Children|    Toy Story (1995)|        49695|
|            Comedy| Pulp Fiction (1994)|        67310|
|             Crime| Pulp Fiction (1994)|        67310|
|       Documentary|Bowling for Colum...|        12280|
|             Drama| Pulp Fiction (1994)|        67310|
|           Fantasy|    Toy Story (1995)|        49695|
|         Film-Noir|L.A. Confidential...|        26836|
|            Horror|Silence of the La...|        63299|
|              IMAX|    Apollo 13 (1995)|        47777|
|           Musical|      Aladdin (1992)|        41842|
|           Mystery|Usual Suspects, T...|       

## Query 17: Find the total number of users watching the same movie, on the same day and time.

In [88]:
# create new dataframe 'same time_user' with the columns of dataframe 'rating'
# grouped by the columns 'movieId', 'year', 'day_of_year' and 'hour',
# and count the number of users who watched the same movie at the same time
same_time = rating_df.select('*').groupby(rating_df["movieId"]).agg(count('userId').alias('viewers'))


In [91]:
rating = rating_df.select(col('userId'),
                      col('movieId'),
                      col('rating'),
                      col('timestamp'),
                      year(col('timestamp')).alias('year'),
                      dayofyear(col('timestamp')).alias('day_of_year'),
                      hour(col('timestamp')).alias('hour'))

In [92]:
# from dataframe 'same_time' select only the rows which have more than one viewer
# and compute the sum of viewers of these rows
same_time = rating.select('*').groupby(rating["movieId"], rating["year"], rating["day_of_year"], rating["hour"]).agg(count('userId').alias('viewers'))


In [93]:
query = same_time.filter(same_time["viewers"] != 1).select('*').agg(sum('viewers').alias('total_viewers')).show()


+-------------+
|total_viewers|
+-------------+
|      4280240|
+-------------+



## Query 18: Find the number of movies, for each category, that users rated as "funny" and with a rating > 3.5.

In [94]:
tag = tag_df.select((col('userId')),
                  col('movieId'),
                  lower(col('tag')).alias('lower_tag'),
                  col('timestamp'))

In [95]:
# execute inner join between 'rating' and 'tag' dataframes
joined = rating_df.join(tag, (rating.userId == tag.userId) & (rating.movieId == tag.movieId), 'inner')


In [96]:
# from dataframe 'joined' select only the rows whose 'lower_tag' column contains the word funny
# and whose 'rating' column is > 3.5
funny_movies = joined.filter((joined['lower_tag'].contains('funny')) & (~joined["lower_tag"].contains('no')) & (joined["rating"] > 3.5))

In [98]:
# because some movies have more than one genres listed,
# we split the column 'genres' with the '|' operator so we can process on each genre separately
# and then use explode so that every distinct genre will be in their own row
movie = movie.withColumn("genres", explode(split("genres", "[|]")))
# execute inner join between 'movie' and 'funny_movies' dataframes
joined = movie.join(funny_movies, ["movieId"], 'inner')

In [99]:
# create new dataframe 'genre' with the columns of dataframe 'joined'
# grouped by the column 'genres'
# and count the number of distinct movies for each genre
genre = joined.select('*').groupby(joined["genres"]).agg(count('title').alias('movies_count'))


In [100]:
query = genre.orderBy(genre["genres"].asc()).show()


+-----------+------------+
|     genres|movies_count|
+-----------+------------+
|     Action|         431|
|  Adventure|         465|
|  Animation|         268|
|   Children|         273|
|     Comedy|        1618|
|      Crime|         276|
|Documentary|          27|
|      Drama|         544|
|    Fantasy|         306|
|  Film-Noir|           3|
|     Horror|         140|
|       IMAX|          74|
|    Musical|          92|
|    Mystery|          93|
|    Romance|         490|
|     Sci-Fi|         197|
|   Thriller|         236|
|        War|          37|
|    Western|          44|
+-----------+------------+

